In [ ]:
import geopandas as gpd

In [ ]:
counties = gpd.read_file("../datasets/WA_County_Boundaries.geojson")
pop_centers = gpd.read_file("../datasets/WSDOT_-_Population_Centers.geojson")

In [ ]:
def find_intersecting_county(polygon, counties_gdf):
    intersecting_counties = counties_gdf[counties_gdf.geometry.contains(polygon)]
    # select counties that intersect the polygon with non-zero area (exclude touching-only)
    mask = counties_gdf.geometry.intersects(polygon) & ~counties_gdf.geometry.touches(polygon)
    intersecting_counties = counties_gdf[mask]
    poly_area = polygon.area
    intersecting_counties = intersecting_counties.copy()

    def _info(row):
        inter = row.geometry.intersection(polygon)
        inter_area = 0.0 if inter.is_empty else inter.area
        pct = (inter_area / poly_area * 100.0) if poly_area > 0 else None
        return {
            "JURISDICT_LABEL_NM": row["JURISDICT_LABEL_NM"],
            "Intersection_Area": inter_area,
            "Pct_of_Polygon": pct,
        }

    # replace the name column with a dict containing name, intersection area, and pct
    intersecting_counties["JURISDICT_LABEL_NM"] = intersecting_counties.apply(_info, axis=1)
    
    # remove entries whose Pct_of_Polygon is less than 0.9 (treat None as 0)
    pct_series = intersecting_counties["JURISDICT_LABEL_NM"].apply(
        lambda d: (d.get("Pct_of_Polygon") if isinstance(d, dict) and d.get("Pct_of_Polygon") is not None else 0.0)
    )
    intersecting_counties = intersecting_counties[pct_series >= 0.9]

    # optional: sort so largest intersections come first
    # intersecting_counties = intersecting_counties.sort_values(by="Pct_of_Polygon", ascending=False)
    return intersecting_counties['JURISDICT_LABEL_NM'].tolist()

In [ ]:
pop_centers['Intersecting_Counties'] = pop_centers.geometry.apply(lambda geom: find_intersecting_county(geom, counties))

In [ ]:
pop_centers['Counties_Count'] = pop_centers['Intersecting_Counties'].apply(len)

In [ ]:
pop_centers['Counties_Count'].value_counts()


In [ ]:
pop_centers[['PlaceName', 'Counties_Count']][pop_centers['Counties_Count'] > 1]